## Benchmarking on dice

Update 1: implement the l1/linf norm in python to measure distance (as in pl code)

*Using the dice-ml package: 

-- it uses its own encoder of categorical/nominal attributes

-- our encoding scheme does not work with dice-ml

-- alternative: all features as numerical BUT could induce problems with decoding (more than one category per feature is active)

-- sources
https://github.com/interpretml/DiCE/blob/8027ebbf696e8b6c9344a889fb1ba4e90ea448d9/docs/source/notebooks/Benchmarking_different_CF_explanation_methods.ipynb
https://interpret.ml/DiCE/notebooks/DiCE_with_private_data.html
https://interpret.ml/DiCE/notebooks/DiCE_getting_started.html

CONSTRAINTS AND DICE

-- specify which features should be varied
-- specify the range in which the features can vary



TODO 

-- run over bigger set of instances, instantiate various constraints and compare with REASONX

-- check why there is a big diff in accuracy (encoding of education (?))

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import sklearn.ensemble 
import matplotlib.pyplot as plt
import pydotplus
from IPython.display import Image
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# local imports
sys.path.append('../src/') # local path

from helper_functions import read_adult, read_give_me_some_credit, read_south_german_credit
from helper_functions import linf_norm_df, l1_norm_df, linf_weights_reasonx, l1_weights_reasonx

import dice_ml

In [2]:
# dataset execution, copied
simplified=False
continuous_only=False

dataset = "adult"

# read dataset
if dataset == "gmsc":
    df, pred_atts, target, df_code = read_give_me_some_credit(continuous_only=continuous_only, simplified=simplified)
if dataset == "sgc":
    df, pred_atts, target, df_code = read_south_german_credit(continuous_only=continuous_only, simplified=simplified)
if dataset == "adult":
    df, pred_atts, target, df_code, nominal_atts, ordinal_atts, continous_atts = read_adult(continuous_only=continuous_only, simplified=simplified, return_feature_type=True)

In [3]:
# encode df
df_encoded_onehot = df_code.fit_transform(df)
# encoded atts names
encoded_pred_atts = df_code.encoded_atts(pred_atts)
df_encoded_onehot.head()

,race_AmerIndianEskimo,race_AsianPacIslander,race_Black,race_Other,race_White,sex_Female,sex_Male,workclass_Federalgov,workclass_Localgov,workclass_Neverworked,...,workclass_Selfempinc,workclass_Selfempnotinc,workclass_Stategov,workclass_Withoutpay,education,age,capitalgain,capitalloss,hoursperweek,class
0,0,0,0,0,1,0,1,0,0,0,...,0,0,1,0,13,39,2174,0,40,0
1,0,0,0,0,1,0,1,0,0,0,...,0,1,0,0,13,50,0,0,13,0
2,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,9,38,0,0,40,0
3,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,7,53,0,0,40,0
4,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,13,28,0,0,40,0


In [4]:
# prepare the dataset for dice
# class must be an integer (and not an object)
df_dice = df.drop([target], axis=1)
df_dice[target] = df_encoded_onehot[target]
df_dice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   race          48842 non-null  object
 1   sex           48842 non-null  object
 2   workclass     48842 non-null  object
 3   education     48842 non-null  object
 4   age           48842 non-null  int64 
 5   capitalgain   48842 non-null  int64 
 6   capitalloss   48842 non-null  int64 
 7   hoursperweek  48842 non-null  int64 
 8   class         48842 non-null  int64 
dtypes: int64(5), object(4)
memory usage: 3.4+ MB


In [5]:
df_target = df_dice[target]

train_dataset, test_dataset, y_train, y_test = train_test_split(df_dice, df_target, test_size=0.3, random_state=42)

x_train = train_dataset.drop(target, axis=1)
x_test = test_dataset.drop(target, axis=1)

x_train.head()

,race,sex,workclass,education,age,capitalgain,capitalloss,hoursperweek
42392,Black,Male,Private,Somecollege,19,0,0,40
14623,White,Male,Private,HSgrad,32,0,0,40
27411,White,Male,Private,Somecollege,43,7688,0,40
1288,White,Male,Private,Bachelors,37,0,0,45
7078,AmerIndianEskimo,Female,Federalgov,Assocvoc,42,0,0,40


In [6]:
d  = dice_ml.Data(dataframe=train_dataset, continuous_features=continous_atts, outcome_name=target)

In [7]:
numerical = continous_atts
categorical = x_train.columns.difference(numerical)

categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

# apply transformation on columns separately
transformations = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations), ('classifier', XGBClassifier(random_state=0))])

model = clf.fit(x_train, y_train)

print("accuracy     ", model.score(x_test,y_test))

accuracy      0.7850952023476421


In [8]:
weights_linf = linf_weights_reasonx(df_encoded_onehot, df_code, ordinal_atts, continous_atts)
weights_l1 = l1_weights_reasonx(df_encoded_onehot, df_code, ordinal_atts, continous_atts)

In [9]:
# Using sklearn backend
m = dice_ml.Model(model=model, backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m, method="random")

In [10]:
instances = 100
n_total_ce = [2, 3, 4, 5]
evaluation_norms = np.zeros((instances, len(n_total_ce), 2))

# DiCE allows tunable parameters proximity_weight (default: 0.5) and diversity_weight (default: 1.0) to handle proximity and diversity respectively.
proximity_weight=0.5
diversity_weight=0

#features_to_vary=["sex", "race", "workclass", "education", "age", "capitalloss", "hoursperweek"]
features_to_vary=["workclass", "capitalgain", "education", "age", "capitalloss", "hoursperweek"]
#features_to_vary = []

for k in range(len(n_total_ce)):
    for i in range(instances):
        if len(features_to_vary) > 0:
            e1 = exp.generate_counterfactuals(x_test[i:i+1], total_CFs=n_total_ce[k], desired_class="opposite", features_to_vary=features_to_vary, proximity_weight=proximity_weight, diversity_weight=diversity_weight)
        else:
            e1 = exp.generate_counterfactuals(x_test[i:i+1], total_CFs=n_total_ce[k], desired_class="opposite", proximity_weight=proximity_weight, diversity_weight=diversity_weight)
        #e1.visualize_as_dataframe(show_only_changes=False)

        # results as dataframe
        s = e1.cf_examples_list[0].final_cfs_df

        # compute l1 and linf norm

        # copy orig data frame
        df_encoding = df.copy()
        full_instance = test_dataset[i:i+1]

        # replace label
        s[target] = np.where(s[target]==0, '<=50K', '>50K')
        full_instance[target] = np.where(full_instance[target]==0, '<=50K', '>50K')

        # append CEs
        # add generated CEs (at top)
        df_encoding = pd.concat([s, df_encoding], ignore_index=False)
        # add original instance (at top)
        df_encoding = pd.concat([full_instance, df_encoding], ignore_index=False)

        #df_encoding.head()
        # transform using orig transformer
        df_test_data_encoded = df_code.transform(df_encoding)

        #df_test_data_encoded.head()
        l1 = []
        linf = []

        for j in range (len(s)):
            # distance original instance - generated CEs
            l1.append(l1_norm_df(df_test_data_encoded.iloc[0,:-1], df_test_data_encoded.iloc[j + 1,:-1], weights_l1))
            linf.append(linf_norm_df(df_test_data_encoded.iloc[0,:-1], df_test_data_encoded.iloc[j + 1,:-1], weights_linf))

        evaluation_norms[i, k, 0] = np.mean(l1)
        evaluation_norms[i, k, 1] = np.mean(linf)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  6.56it/s]
/tmp/ipykernel_88739/967433733.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_instance[target] = np.where(full_instance[target]==0, '<=50K', '>50K')
100%|██████████| 1/1 [00:00<00:00,  5.47it/s]
/tmp/ipykernel_88739/967433733.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_instance[target] = np.where(full_instance[target]==0, '<=50K', '>50K')
100%|██████████| 1/1 [00:00<00:00,  7.80it/s]
/tmp/ipykernel_88739/967433733.py:32: SettingWithCopyW

In [11]:
# average distance l1/linf between original data point and CE, for "instances" number of data instances
print(np.mean(evaluation_norms, axis = 0))
#print(np.std(evaluation_norms, axis = 0))

[[0.97564883 0.68047676]
 [0.99455834 0.68323592]
 [0.9828406  0.68682984]
 [0.97392085 0.66797102]]
